<a href="https://colab.research.google.com/github/Satwikram/Large-Language-Models-Implementation/blob/main/Llama/Using%20llama-2%20and%20langchain%20for%20question%20answering%20on%20your%20own%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram